In [2]:
guideline = (
    "Act as an ULTIMATE Python Coding Expert. Follow instructions given below STRICTLY.\n"
    "Input: A programming question in English with function name and parameters and test cases list.\n"
    "Output: Only the correct Python code inside a fenced code block.\n\n"

    "Format:\n"
    "```python\n"
    "<code here>\n"
    "```\n\n"

    "Here is example question:\n"
    """Instruction: Write a python program to find the sum of first n natural numbers.\n
    Example:\n
    sum_series(number):\n\n
    """
    "Output:\n"
    "```python\n"
    "import math\n"
    "def sum_series(number):\n"
    "    total = 0\n"
    "    total = (number * (number + 1)) / 2\n"
    "    return total\n"
    "```\n\n"

    "Rules:\n"
    "- DEEPLY think about all possible solutions step by step and ALWAYS provide the CORRECT code.\n"
    "- Handle all possible edge cases ROBUSTLY and the code MUST PASS all the test cases.\n"
    "- Preserve the function name and parameters exactly as provided.\n"
    "- Always use return statements to output results.\n"
    "- Write Python code without using classes. Only use functions and variables.\n"
    "- If the function needs imports, import required libraries.\n"
    "- Do not include any text, explanation, comments, or docstrings.\n"
    "- Do not write code for reading input (no input()) or printing output (no print()).\n"
)

In [3]:
guideline

'Act as an ULTIMATE Python Coding Expert. Follow instructions given below STRICTLY.\nInput: A programming question in English with function name and parameters and test cases list.\nOutput: Only the correct Python code inside a fenced code block.\n\nFormat:\n```python\n<code here>\n```\n\nHere is example question:\nInstruction: Write a python program to find the sum of first n natural numbers.\n\n    Example:\n\n    sum_series(number):\n\n\n    Output:\n```python\nimport math\ndef sum_series(number):\n    total = 0\n    total = (number * (number + 1)) / 2\n    return total\n```\n\nRules:\n- DEEPLY think about all possible solutions step by step and ALWAYS provide the CORRECT code.\n- Handle all possible edge cases ROBUSTLY and the code MUST PASS all the test cases.\n- Preserve the function name and parameters exactly as provided.\n- Always use return statements to output results.\n- Write Python code without using classes. Only use functions and variables.\n- If the function needs impo

## Step 0 - Installing Librariess

In [4]:
!pip install -q --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install -q sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install -q --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 39.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires n

## Step 1 - Choose a Base Model

In [5]:
import unsloth
from unsloth import FastModel
import torch
from transformers import AutoConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-09 12:38:41.174223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757421521.544594      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757421521.652365      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


## Step 2 - Play with the Base Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

# Load Qwen model & tokenizer
model_name = "Qwen/Qwen2.5-Coder-14B-Instruct"
# Use 4-bit quantization for QLoRA

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16  # bfloat16 is risky on T4
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()},
    torch_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # ✅ Optional: avoid padding issues

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
# !pip install -q transformers sentencepiece

In [ ]:
bangla_prompt = "একটি স্ট্রিংয়ের শেষে একটি সংখ্যা পরীক্ষা করার জন্য একটি ফাংশন লিখুন।"


# Final prompt for Qwen
full_prompt = f"{bangla_prompt.strip()}\n\n{guideline}"

# Apply chat template
chat_text = tokenizer.apply_chat_template(
    [{"role": "user", "content": full_prompt}],
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize & generate
inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.3,
        top_p=0.95,
        top_k=64,
        pad_token_id=tokenizer.pad_token_id
    )

# Decode generated part
gen_ids = outputs[0][inputs["input_ids"].shape[1]:]
response = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

print(response)


📘 Translated: Write a function to test a number at the end of a string.
Instruction: Write a function to test if a number is at the end of a string.

    Example:

    is_number_at_end(s):

    Test Cases:
    - is_number_at_end("hello123") -> True
    - is_number_at_end("hello") -> False
    - is_number_at_end("123") -> True
    - is_number_at_end("hello123world") -> False
    - is_number_at_end("") -> False

Output:
```python
import re
def is_number_at_end(s):
    return bool(re.search(r'\d+$', s))
```


In [12]:
import pandas as pd
import torch
from tqdm.auto import tqdm

# Load dev set
df = pd.read_csv("/kaggle/input/blp-test-data-foyez/test-foyez.csv")  # Columns: id, instruction
out_df['test_list'] = df['test_list']  # Carry over test_list
df = out_df  # df now contains: id, translated_prompt, test_list


# Disable truncation for strings
pd.set_option("display.max_colwidth", None)
df.head()

assert {"id", "instruction", "test_list"}.issubset(df.columns)


# Generate responses
responses = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="🔁 Generating"):
    full_prompt = (
        f"{guideline}\n\n"
        f"Here is the programming question: {row['instruction'].strip()}\n\n"
        f"Test cases: {row['test_list']} \n"
        f"Now write a python code that will pass all of this test cases."
    )

    chat_text = tokenizer.apply_chat_template(
        [{"role": "user", "content": full_prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )

    # print(chat_text)
    # break

    inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.6,
            top_p=0.95,
            top_k=50,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
        )

    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    responses.append(gen_text)

# sanity check
assert len(df) == len(responses), "Mismatch between input and output rows!"

# Save
out_df = pd.DataFrame({"id": df["id"], "response": responses})
out_df.to_json("submission.json", orient="records", force_ascii=False, indent=2)
print(f"✅ Wrote submission.json with {len(out_df)} rows")

# Show top 10 entries from the saved JSON
with open("submission.json", "r", encoding="utf-8") as f:
    import json
    data = json.load(f)
    print("🔹 Top 10 JSON entries:")
    for entry in data[:10]:
        print(entry)

🔁 Generating:   0%|          | 0/500 [00:00<?, ?it/s]

✅ Wrote submission.json with 500 rows
🔹 Top 10 JSON entries:
{'id': 1, 'response': '```python\ndef remove_Occ(s, ch):\n    first_occurrence = s.find(ch)\n    if first_occurrence != -1:\n        s = s[:first_occurrence] + s[first_occurrence+1:]\n    \n    last_occurrence = s.rfind(ch)\n    if last_occurrence != -1:\n        s = s[:last_occurrence] + s[last_occurrence+1:]\n    \n    return s\n```'}
{'id': 2, 'response': '```python\ndef sort_matrix(M):\n    return sorted(M, key=sum)\n```'}
{'id': 3, 'response': '```python\nfrom collections import Counter\n\ndef count_common(words):\n    word_counts = Counter(words)\n    most_common_words = word_counts.most_common()\n    return most_common_words\n```'}
{'id': 4, 'response': '```python\nimport math\n\ndef find_Volume(a, b, c):\n    s = (a + b + c) / 2\n    volume = (s * (s - a) * (s - b) * (s - c)) ** 0.5\n    return volume\n```'}
{'id': 5, 'response': '```python\ndef split_lowerstring(text):\n    result = []\n    for i in range(1, len(text

In [13]:
len(responses)

500

In [14]:
out_df.shape

(500, 2)

In [15]:
df.shape

(500, 3)

##  Preparing Submission File

In [16]:
import json, os, re, zipfile

SUB_PATH = "submission.json"

def file_format_check(path: str) -> bool:
    # name + extension
    if os.path.basename(path) != "submission.json":
        print("Error: File name must be exactly 'submission.json'")
        return False
    if not path.lower().endswith(".json"):
        print("Error: File must have .json extension")
        return False

    # must be valid JSON (not JSONL) and root must be a list
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {e}")
        print("Note: The file must be in proper JSON format (not JSONL)")
        return False

    if not isinstance(data, list):
        print("Error: The root element should be a list of objects")
        return False

    # each item: dict with ONLY keys {'id','response'}; id=int; response=str
    for idx, item in enumerate(data):
        if not isinstance(item, dict):
            print(f"Error: Item at index {idx} is not a dictionary")
            return False
        keys = set(item.keys())
        if keys != {"id", "response"}:
            print(f"Error: Item at index {idx} must contain only keys 'id' and 'response', found: {keys}")
            return False
        if not isinstance(item["id"], int):
            print(f"Error: 'id' field at index {idx} must be an integer")
            return False
        if not isinstance(item["response"], str):
            print(f"Error: 'response' field at index {idx} must be a string")
            return False

    print("Format check passed successfully!")
    return True

# ---------- Load, compute per-item validity, blank invalids, save, zip ----------
# Load JSON list
with open(SUB_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

n = len(data)
fence_pat = re.compile(r"^```python[\s\S]*```$", re.MULTILINE)

valid_format = []
valid_fence  = []
valid_both   = []

# Per-item validation mirrors file checker semantics
def item_format_ok(item):
    return (
        isinstance(item, dict)
        and set(item.keys()) == {"id", "response"}
        and isinstance(item["id"], int)
        and isinstance(item["response"], str)
    )

for item in data:
    vfmt = item_format_ok(item)
    vf   = bool(fence_pat.match(item["response"])) if vfmt else False
    valid_format.append(vfmt)
    valid_fence.append(vf)
    valid_both.append(vfmt and vf)

# After computing valid_format, valid_fence, valid_both
for i, item in enumerate(data):
    if not valid_format[i]:
        print(f"❌ Format Error at index {i}: {item}")
    elif not valid_fence[i]:
        print(f"❌ Fencing Error at index {i} (id={item.get('id')}):")
        print(item["response"])
        print("-" * 50)


# Report stats
nf = sum(valid_fence)
nm = sum(valid_format)
nb = sum(valid_both)
den = max(n, 1)
print(f"Fencing valid: {nf}/{n} ({nf*100.0/den:.1f}%)")
print(f"Format valid:  {nm}/{n} ({nm*100.0/den:.1f}%)")
print(f"Both valid:    {nb}/{n} ({nb*100.0/den:.1f}%)")

# Strict policy: blank responses that fail ANY check
for i, ok in enumerate(valid_both):
    if not ok and isinstance(data[i], dict) and "response" in data[i]:
        data[i]["response"] = ""

# Overwrite submission.json (id+response only)
with open(SUB_PATH, "w", encoding="utf-8") as f:
    json.dump(
        [{"id": item["id"], "response": item["response"]} for item in data],
        f, ensure_ascii=False, indent=2
    )
print("✅ Updated submission.json after checks (invalid responses blanked).")

# Final file-level check (should pass)
_ = file_format_check(SUB_PATH)

# Zip as submission.zip (Jupyter-friendly, no shell commands)
with zipfile.ZipFile("submission.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(SUB_PATH)
print("📦 Created submission.zip containing submission.json.")


Fencing valid: 500/500 (100.0%)
Format valid:  500/500 (100.0%)
Both valid:    500/500 (100.0%)
✅ Updated submission.json after checks (invalid responses blanked).
Format check passed successfully!
📦 Created submission.zip containing submission.json.


In [17]:
# with open("/kaggle/working/submission.json", "r", encoding="utf-8") as f:
#     content = f.read()

# print(content)

# Submit the submission.zip file in CodaBench

### Save the NEW model...if it's good :)

In [18]:
# model.save_pretrained("New_Model")  # Local saving
# tokenizer.save_pretrained("New_Model")